In [1]:
%pwd

'/Users/winston/mygitrep/sys23/demo'

# Factor: `mkt_dvg_min_amt`

Market Divergence of Minute Amount

成交额市场分歧度择时：

1. 价格高位【当日实现收益相对过去20日日收益均值处于较高水平】；
2. 市场分化【分钟收益全市场趋同时】

收益率 ~ 股价成交额与其他股票成交额的关联性

- 股价高位时：关联越高越好，“随波逐流”
    - 价格高位由市场趋势带动，持续
    - 投资者意见分歧，结束
- 市场分化不明显时：关联越低越好，“孤雁出群”
    - 市场没有热点，独立异动酝酿新热点
- 等权合成：“水中行舟”
    - 月度选股，RankIC 9.36e-2, Rank ICIR -4.95, LS Yearly Return 36.24\%, IR 4.40, Monthly WR 86.67%
    - 剔除常用风格因子
    - 成份股：CSI300, CSI500, CSI1000 多头年化收益 9.48\%, 9.71\%, 18.51\%

[《方正证券-个股成交额的市场跟随性与“水中行舟”因子》](../idea/docs/个股成交额的市场跟随性与“水中行舟”因子——多因子选股系列研究之九.pdf)


## 数据准备

## 因子构造


> - 【】：描述经济直觉
> - 『』：提出改进方向


### 随波逐流

(1) $t$ 日，合理收益率【20天平均表现】『参数20日窗口，可适应到价格稳定程度：价格波动快，窗口尽量短』

$$
\tilde{R}_t = \frac{1}{2} \sum_{i=t-19}^{t} 
\left(\frac{P_i^c}{P_i^o} - 1\right)
$$

(2) $i$ 分钟，相对开盘收益率【反应当前股价趋势：上行or下行】『需要分钟频k线；更细频率；不等权』
$$
\tilde{R}_t^{i} = \frac{P_t^i}{P_t^o} - 1,\ i=1,2,\dots,240
$$
(3) $t$ 日，高低额差【成交发生于高位or低位】『参数 $\epsilon$ 判别趋势偏离度：研报取0；宜由窗口内日收益标准差确定 』
$$
\delta_t = \frac{1}{\text{MV}_t}
\sum_{i=1}^{240}{
    \text{Amt}_t^i 
    \times
    \left(
        \textbf{1}\{
            \tilde{R}_t^i > 
            \tilde{R}_t\ + \epsilon
        \}
        -
        \textbf{1}\{
            \tilde{R}_t^i < 
            \tilde{R}_t - \epsilon
        \}
    \right)
}
$$
(4) 月底换仓日 $T$，股 $(s)$ 的随波逐流因子【20日窗口期内趋势与大盘的同质性】『20期是否过短？』
$$
F_T^{(k)} = Mean_{o\not=s}(Abs(Corr_{t\in[T-19,T]}^{Spearman}(
	\delta_t^{(s)},
	\delta_t^{(o)}
))
$$
(5) 全A；市值、行业正交化；测试区间201301-202301；10分组



### 孤雁出群

(1) 分钟收益率
$$
R_i = \frac{P_i}{P_{i-1}}-1,\ i=1,2,\dots,240
$$
(2) 分钟 $i$，市场分化度【衡量截面分钟收益差异】『观察分布，剔除对极端个股』
$$
D_i=StdDev_{(s)}(R_i^{(s)})
$$
(3) 不分化时刻【日内截面收益差异较小的（一半）时刻】『设置不分化时刻每天所占比例；再由20日（窗口期）$\{\bar{D}_{t}\}$ 标准差确定 $\epsilon$ ，当天截面分化越严重，则该天的不分化时刻占比越少』
$$
\mathbb{D}=\{i: D_i < \bar{D} + \lambda \sigma\},\\
\bar{D}=Quantile_{i\in[1,240]}^{0.5+\eta}(D_i),\\
\eta\in(-0.5,0.5)
\\ \lambda>0,\\
\sigma =StdDev_{t\in[ct-19,ct]}(\bar{D}_t)
$$
(4) “日孤雁出群”【市场分化不明显时，交易额与市场的关联性；皮尔逊相关，提高交易活跃的分钟所占比重（因为大部分时间交易不活跃）】
$$
f_t^{(s)} = Mean_{o\not=s}(Abs(Corr_{i\in\mathbb{D}}^{Pearson}(\text{Amt}_i^{(s)}, \text{Amt}_i^{(o)})))
$$
(5) 月换仓日“孤雁出群”：“月均孤雁出群” + “月稳孤雁出群”
$$
F_T^{m,(s)}= Mean_{t\in[T-19,T]}(f_t^{(s)})\\
F_T^{sd,(s)} = StdDev_{t\in[T-19,T]}(f_t^{(s)})\\
F_T^{(s)} = F_T^{m,(s)} + F_T^{sd,(s)}
$$


### 水中行舟

- 随波逐流和孤雁出群相关性接近0『既然不相关，那就作为独立的两个因子使用』
- 等权合成『可以不等权，但需要逻辑』


## 市场回测